# 中文情感分析
Chinese sentiment analysis

### 载入所需的库

In [3]:
import sys
# reload(sys)
# sys.setdefaultencoding('utf8')
import numpy as np 
import pandas as pd
import jieba

from sklearn.model_selection import train_test_split
from sklearn.externals import joblib  # 可以把数据用二进制形式存储，读取
from sklearn.svm import SVC

from gensim.models.word2vec import Word2Vec


### 加载数据，做预处理（分词），切分训练集和测试集
下面内容均为分步解析，函数load_file_and_preprocessing才是主体。

In [6]:
neg = pd.read_excel('data/neg.xls', header=None, index=None)
pos = pd.read_excel('data/pos.xls', header=None, index=None)

neg.head()

,0
0,做为一本声名在外的流行书，说的还是广州的外企，按道理应该和我的生存环境差不多啊。但是一看之下...
1,作者有明显的自恋倾向，只有有老公养不上班的太太们才能像她那样生活。很多方法都不实用，还有抄袭...
2,作者完全是以一个过来的自认为是成功者的角度去写这个问题，感觉很不客观。虽然不是很喜欢，但是，...
3,作者提倡内调，不信任化妆品，这点赞同。但是所列举的方法太麻烦，配料也不好找。不是太实用。
4,作者的文笔一般，观点也是和市面上的同类书大同小异，不推荐读者购买。


In [7]:
pos.head()

,0
0,做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一...
1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...
2,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...
3,作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...
4,作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...


In [9]:
print(neg.shape)
print(pos.shape)

(10428, 1)
(10677, 1)


In [15]:
' '.join(list(jieba.cut(pos[0][0])))

'做 父母 一定 要 有 刘墉 这样 的 心态 ， 不断 地 学习 ， 不断 地 进步 ， 不断 地 给 自己 补充 新鲜血液 ， 让 自己 保持 一颗 年轻 的 心 。 我 想 ， 这 是 他 能 很 好 的 和 孩子 沟通 的 一个 重要 因素 。 读 刘墉 的 文章 ， 总能 让 我 看到 一个 快乐 的 平易近人 的 父亲 ， 他 始终 站 在 和 孩子 同样 的 高度 ， 给 孩子 创造 着 一个 充满 爱 和 自由 的 生活 环境 。 很 喜欢 刘墉 在 字里行间 流露出 的 做 父母 的 那种 小 狡黠 ， 让 人 总是 忍俊不禁 ， 父母 和 子女 之间 有时候 也 是 一种 战斗 ， 武力 争斗 过于 低级 了 ， 智力 较量 才 更 有 趣味 。 所以 ， 做 父母 的 得 加把劲 了 ， 老 思想 老 观念 注定 会 一败涂地 ， 生命不息 ， 学习 不止 。 家庭教育 ， 真的 是 乐在其中 。'

In [18]:
y = np.concatenate((np.ones(len(pos)), np.zeros(len(neg))))
y

array([1., 1., 1., ..., 0., 0., 0.])

以上内容均为分步解析，函数load_file_and_preprocessing才是主体。

In [17]:
def load_file_and_preprocessing():
    # 读取neg, pos内容
    neg = pd.read_excel('data/neg.xls', header=None, index=None)
    pos = pd.read_excel('data/pos.xls', header=None, index=None)
    
    # 分词
    cw = lambda x: list(jieba.cut(x))
    pos['words'] = pos[0].apply(cw)
    neg['words'] = neg[0].apply(cw)
    
    # 生成对应的pos:1 , neg: 0 的label
    y = np.concatenate((np.ones(len(pos)), np.zeros(len(neg))))
    new_df = np.concatenate((pos['words'], neg['words']))
    
    x_train, x_test, y_train, y_test = train_test_split(new_df, y, test_size=0.2)
    
    np.save('svm_data/y_train.npy', y_train)
    np.save('svm_data/y_test.npy', y_test)
    
    return x_train, x_test

In [33]:
x_train, x_test = load_file_and_preprocessing()

In [34]:
print('x_train\'s shape: ', x_train.shape)
print('x_test\'s shape: ', x_test.shape)

x_train's shape:  (16884,)
x_test's shape:  (4221,)


In [47]:
print(np.load('svm_data/y_train.npy'))
print(np.load('svm_data/y_train.npy').shape)

[1. 0. 1. ... 0. 0. 1.]
(16884,)


### 对每个句子的所有词向量取均值，来生成一个句子的vector

In [21]:
def build_sentence_vector(text, size, imdb_w2v):
    vec = np.zeros(size).reshape((1,size))
    count = 0
    for word in text:
        try:
            vec += imdb_w2v.wv[word].reshape((1, size))
            count += 1
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

### 计算词向量

In [87]:
# 根据x_train的文本信息，训练word2vec的模型
imdb_w2v = Word2Vec(size=300, min_count=10)
imdb_w2v.build_vocab(x_train)
imdb_w2v.train(x_train, total_examples=imdb_w2v.corpus_count, epochs=imdb_w2v.epochs)

(3642161, 5392895)

In [88]:
' '.join(x_train[0])

'技术 区里 唯一 的 四星级 酒店 , 外观 不起眼 , 但 服务 感觉 很 不错 , 尤其 ?'

In [89]:
imdb_w2v.wv['技术'].reshape((1,300)).shape

(1, 300)

In [91]:
build_sentence_vector(x_train[0], 300, imdb_w2v).shape

/Users/zoe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(1, 300)

In [96]:
def get_train_vecs(x_train, x_test):
    n_dim = 300
    
    # 初始化模型和词表
    imdb_w2v = Word2Vec(size=n_dim, min_count=10)
    imdb_w2v.build_vocab(x_train)
    
    # 在评论训练集上建模
    imdb_w2v.train(x_train, total_examples=imdb_w2v.corpus_count, epochs=imdb_w2v.epochs)
    
    train_vecs = np.concatenate([build_sentence_vector(z, n_dim, imdb_w2v) for z in x_train])
    
    np.save('svm_data/train_vecs.npy', train_vecs)
    
    print(train_vecs.shape)
    
    # 在测试集上训练
    imdb_w2v.train(x_test, total_examples=imdb_w2v.corpus_count, epochs=imdb_w2v.epochs)
    imdb_w2v.save('svm_data/w2v_model/w2v_model.pkl')
    
    # Build test tweet vectors then scale
    test_vecs = np.concatenate([build_sentence_vector(z, n_dim, imdb_w2v) for z in x_test])
    
    np.save('svm_data/test_vecs.npy', test_vecs)
    
    print(test_vecs.shape)

In [97]:
get_train_vecs(x_train, x_test)

/Users/zoe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(16884, 300)
(4221, 300)


In [24]:
def get_data():
    train_vecs = np.load('svm_data/train_vecs.npy')
    y_train = np.load('svm_data/y_train.npy')
    test_vecs = np.load('svm_data/test_vecs.npy')
    y_test = np.load('svm_data/y_test.npy')
    return train_vecs, y_train, test_vecs, y_test

In [99]:
train_vecs, y_train, test_vecs, y_test = get_data()

In [101]:
print('train_vecs shape: ', train_vecs.shape)
print('test_vecs shape: ', test_vecs.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

train_vecs shape:  (16884, 300)
test_vecs shape:  (4221, 300)
y_train shape:  (16884,)
y_test shape:  (4221,)


### 训练SVM模型

In [102]:
def svm_train(train_vecs, y_train, test_vecs, y_test):
    clf = SVC(kernel='rbf', verbose=True)
    clf.fit(train_vecs, y_train)
    joblib.dump(clf, 'svm_data/svm_model/model.pkl')
    print(clf.score(test_vecs, y_test))

In [103]:
svm_train(train_vecs, y_train, test_vecs, y_test)

[LibSVM]0.7825159914712153


### 构建待预测句子的向量

In [26]:
def get_predict_vecs(words):
    n_dim = 300
    imdb_w2v = Word2Vec.load('svm_data/w2v_model/w2v_model.pkl')
    train_vecs = build_sentence_vector(words, n_dim, imdb_w2v)
    
    return train_vecs

In [104]:
get_predict_vecs(list(jieba.cut('我们是中国人')))

/Users/zoe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([[-2.34407905e-01,  1.44382074e-01,  1.88170485e-01,
        -4.21376348e-01, -6.39175661e-01,  2.31335017e-01,
        -7.87728578e-02, -5.26599847e-01, -5.63985065e-01,
        -1.90347357e-01,  2.71049142e-03, -3.21473327e-01,
        -5.21203991e-01, -2.74984868e-01,  4.34153784e-01,
        -3.01018286e-01,  1.47170902e-01,  2.03365717e-01,
        -8.27119797e-01, -1.98496759e-01,  7.15714865e-02,
        -9.73958895e-02,  4.59843230e-01, -2.05852618e-01,
        -2.52373662e-01, -3.71734537e-02,  1.56005062e-02,
        -2.32473850e-01,  9.26987249e-02, -2.94366710e-01,
        -1.99573599e-01,  3.41328844e-01,  5.03863804e-02,
         8.55444919e-02, -2.77665779e-01,  3.52835642e-01,
         9.58216488e-02, -3.48808561e-01,  1.91210520e-01,
        -3.92713197e-01, -3.90093991e-01,  5.72854578e-02,
        -1.17767282e-01,  1.71530101e-01, -6.02068968e-01,
        -3.68573114e-01, -1.26361307e-01,  1.14220306e-02,
        -6.66302567e-01,  2.70749770e-01, -3.52008369e-0

### 对单个句子进行情感分析

In [107]:
def svm_predict(string):
    words = jieba.lcut(string)
    words_vecs = get_predict_vecs(words)
    clf = joblib.load('svm_data/svm_model/model.pkl')
    
    result = clf.predict(words_vecs)
    
    if int(result[0]) == 1:
        print(string, 'postive')
    else:
        print(string, 'negative')

In [108]:
string = '电池充完了电连手机都打不开.简直烂的要命.真是金玉其外,败絮其中!连5号电池都不如'
svm_predict(string)

电池充完了电连手机都打不开.简直烂的要命.真是金玉其外,败絮其中!连5号电池都不如 negative


/Users/zoe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
